# 🥋 Lekcja 31: Mixed Precision (AMP) & GradScaler

Standardowo PyTorch używa `float32` (4 bajty na liczbę).
Karty NVIDIA (Volta, Turing, Ampere) mają **Tensor Cores**, które liczą macierze w `float16` (2 bajty) niesamowicie szybko.

**Problem z FP16:**
Zakres liczb jest mały.
*   Najmniejsza liczba dodatnia w FP16 to ok. `6e-5`.
*   Gradienty w sieciach często są rzędu `1e-7`. W FP16 stają się zerem (**Underflow**). Sieć przestaje się uczyć.

**Rozwiązanie (AMP Pipeline):**
1.  **Autocast:** PyTorch automatycznie decyduje, które operacje (Conv, MatMul) zrobić w FP16 (szybkie), a które (Softmax, Sum) zostawić w FP32 (stabilne).
2.  **GradScaler:** Mnoży Loss przez dużą liczbę (np. 65536), żeby "nadmuchać" małe gradienty, by nie zniknęły w FP16. Przed aktualizacją wag dzieli je z powrotem.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

# Sprawdźmy sprzęt
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("⚠️ Brak GPU. AMP zadziała (bfloat16 na CPU), ale zysk będzie mniejszy.")

# Symulacja problemu Underflow
small_grad = torch.tensor(1e-5, dtype=torch.float32)
print(f"Liczba w FP32: {small_grad.item():.8f}")
print(f"Liczba w FP16: {small_grad.half().item():.8f}") 
# 1e-5 w FP16 jest bezpieczne, ale 1e-8 stałoby się zerem!

✅ GPU: NVIDIA GeForce RTX 4060
Liczba w FP32: 0.00001000
Liczba w FP16: 0.00001001


## Wzorzec Projektowy AMP

Kod treningowy zmienia się minimalnie.
Zamiast:
```python
loss = criterion(model(x), y)
loss.backward()
optimizer.step()
```
Robimy:
```python
with torch.autocast(device_type=device, dtype=torch.float16):
    loss = criterion(model(x), y)

scaler.scale(loss).backward() # Skalowanie + Backward
scaler.step(optimizer)        # Odskalowanie + Update
scaler.update()               # Aktualizacja współczynnika skali
```

In [7]:
# Prosty model, ale z dużymi macierzami, żeby poczuć różnicę w pamięci
model = nn.Sequential(
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Linear(4096, 4096)
).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

# Inicjalizacja Skalera (Nowoczesne API PyTorch 2.x)
# Zamiast torch.cuda.amp.GradScaler, używamy torch.amp.GradScaler
# Pierwszy argument to typ urządzenia ('cuda').
scaler = torch.amp.GradScaler('cuda', enabled=(device == "cuda"))

# Dane
data = torch.randn(64, 4096, device=device)
target = torch.randn(64, 4096, device=device)

print("Setup gotowy. Skaler zainicjowany (Nowe API bez ostrzeżeń).")

Setup gotowy. Skaler zainicjowany (Nowe API bez ostrzeżeń).


## Pętla Treningowa z AMP

Zwróć uwagę na `scaler.update()`. Skaler jest inteligentny:
*   Jeśli wykryje `NaN` lub `Inf` (co oznacza, że skala była za duża i nastąpił Overflow) -> **Pominie ten krok** (nie zaktualizuje wag) i zmniejszy skalę.
*   Jeśli przez X kroków nie ma błędów -> Zwiększy skalę, żeby zyskać na precyzji.

In [8]:
print(f"Startowa skala: {scaler.get_scale()}")

start = time.time()
for step in range(10):
    optimizer.zero_grad()
    
    # 1. Autocast (Tu dzieje się magia mieszania typów)
    # dtype=torch.float16 dla GPU NVIDIA
    with torch.autocast(device_type=device, dtype=torch.float16):
        output = model(data)
        loss = (output - target).pow(2).mean() # MSE
    
    # 2. Backward ze skalowaniem
    # Zamiast loss.backward()
    scaler.scale(loss).backward()
    
    # 3. Step ze skalowaniem
    # Zamiast optimizer.step()
    # To odskalowuje gradienty (dzieli przez scale factor) przed aktualizacją wag
    scaler.step(optimizer)
    
    # 4. Aktualizacja samej skali
    scaler.update()
    
    if step % 2 == 0:
        print(f"Krok {step}: Loss={loss.item():.4f}, Skala={scaler.get_scale()}")

print(f"Czas: {time.time() - start:.4f}s")

Startowa skala: 65536.0
Krok 0: Loss=1.0529, Skala=65536.0
Krok 2: Loss=1.0524, Skala=65536.0
Krok 4: Loss=1.0518, Skala=65536.0
Krok 6: Loss=1.0513, Skala=65536.0
Krok 8: Loss=1.0507, Skala=65536.0
Czas: 0.4104s


## Pułapka: Gradient Clipping i Unscale

Co jeśli używasz `clip_grad_norm_` (Lekcja 32)?
Nie możesz przyciąć gradientów, które są przeskalowane (pomnożone przez 65536), bo to bez sensu.
Musisz je najpierw **ręcznie odskalować**.

Kolejność jest krytyczna:
1.  `scaler.scale(loss).backward()`
2.  `scaler.unscale_(optimizer)`  <-- WAŻNE
3.  `clip_grad_norm_(...)`
4.  `scaler.step(optimizer)`
5.  `scaler.update()`

In [9]:
# Demonstracja z Clippingiem
optimizer.zero_grad()

with torch.autocast(device_type=device, dtype=torch.float16):
    output = model(data)
    loss = (output - target).pow(2).mean()

# Backward
scaler.scale(loss).backward()

# --- MANEWR Z CLIPPINGIEM ---
# Musimy najpierw odskalować gradienty w miejscu, żeby policzyć ich prawdziwą normę
scaler.unscale_(optimizer)

# Teraz gradienty są normalnymi liczbami FP32, możemy je ciąć
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# Step (skaler wie, że już zrobiliśmy unscale, więc nie zrobi tego drugi raz)
scaler.step(optimizer)
scaler.update()

print("Krok z Clippingiem wykonany pomyślnie.")

Krok z Clippingiem wykonany pomyślnie.


## 🥋 Black Belt Summary

1.  **Zawsze używaj AMP na GPU.** To darmowa wydajność. Nie ma powodu, by trenować w czystym FP32 (chyba że masz bardzo specyficzne problemy numeryczne).
2.  **`bfloat16` (Brain Float):** Na najnowszych kartach (Ampere A100, Hopper H100) zamiast `float16` można używać `bfloat16`. Ma on taki sam zakres jak `float32`, tylko mniejszą precyzję. Dzięki temu **nie potrzebuje GradScalera**! (Wystarczy samo `autocast`).
3.  **Oszczędność:** Dzięki AMP, tensory aktywacji (zapisywane do backwardu) zajmują połowę miejsca. Możesz podwoić Batch Size.